1. 사용환경 준비

In [ ]:
import os
from langchain.chat_models import ChatOpenAI  # ChatGPT 모델
from langchain.embeddings.openai import OpenAIEmbeddings  # OpenAI 임베딩 모델
from langchain.vectorstores import FAISS  # 문서 검색을 위한 FAISS 라이브러리
from langchain.prompts import ChatPromptTemplate  # 대화용 프롬프트 템플릿
from langchain_community.vectorstores import FAISS  # FAISS를 추가
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain_core.runnables import RunnablePassthrough  # 중간 결과를 출력할 수 있는 클래스
from langchain_openai import ChatOpenAI  # OpenAI 모델
from getpass import getpass  # 비밀번호처럼 안전하게 API 키를 입력 받기 위해 사용

# OpenAI API 키를 안전하게 입력받습니다.
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")  # OpenAI API 키를 입력받고 환경변수로 설정합니다.

# ChatGPT 모델 초기화. 'gpt-4o-mini' 모델을 사용합니다.
model = ChatOpenAI(model="gpt-4o-mini")

from langchain.document_loaders import PyPDFLoader  # PDF 파일을 읽을 수 있게 해주는 라이브러리입니다.

# PDF 파일을 로드합니다. (경로 입력)
loader = PyPDFLoader("C:/Users/USER/Desktop/RAG과제/AI챗봇/2024 한권으로 OK 주식과 세금.pdf")

# PDF 파일을 페이지별로 로드합니다.
docs = loader.load()

from langchain.text_splitter import CharacterTextSplitter  # 긴 텍스트를 잘라주는 라이브러리

# 문서 텍스트를 쪼갭니다. chunk_size는 잘라낼 크기, chunk_overlap은 겹치는 부분의 크기입니다.
text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 두 개의 줄바꿈이 있을 때 문서를 나눕니다.
    chunk_size=100,  # 한 조각의 크기를 100으로 설정합니다.
    chunk_overlap=10,  # 조각이 겹치는 부분을 10으로 설정합니다.
    length_function=len,  # 텍스트의 길이를 기준으로 잘라냅니다.
    is_separator_regex=False,  # 줄바꿈을 정규식으로 처리하지 않도록 설정합니다.
)

# 문서를 잘라서 여러 조각으로 만듭니다.
splits = text_splitter.split_documents(docs)

# OpenAI 임베딩 모델을 초기화합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 문서와 임베딩을 결합하여 벡터 스토어(검색할 수 있는 데이터베이스)를 만듭니다.
vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

# 벡터 스토어에서 유사한 문서를 찾는 retriever를 생성합니다.
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 프롬프트 파일을 불러오는 함수입니다. 
def load_prompt_from_file(filename):
    """주어진 파일에서 프롬프트를 로드"""
    try:
        # 파일을 읽어옵니다.
        with open(filename, 'r', encoding='utf-8') as file:
            return file.read()  # 파일의 내용을 반환합니다.
    except FileNotFoundError:
        # 파일을 찾을 수 없을 때 오류 메시지 출력
        print(f"파일을 찾을 수 없습니다: {filename}")
        return None  # 파일을 찾을 수 없으면 None을 반환

# 사용자에게 사용할 프롬프트를 선택하라고 요청합니다.
prompt_option = input("1, 2, 3 중 하나를 골라주세요").strip()

# 사용자가 선택한 번호에 맞는 프롬프트 파일 경로를 설정합니다.
if prompt_option == '1':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt1.txt"
elif prompt_option == '2':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt2.txt"
elif prompt_option == '3':
    filename = "C:/Users/USER/Desktop/RAG과제/AI챗봇/Prompts/prompt3.txt"
else:
    print("잘못된 번호를 입력하셨습니다.")  # 잘못된 번호가 입력된 경우

# 선택한 프롬프트 파일을 로드합니다.
file_content = load_prompt_from_file(filename)
print(file_content)  # 로드한 프롬프트를 출력합니다.

# 프롬프트 템플릿을 정의합니다. 시스템과 사용자의 입력을 포함한 메시지로 구성됩니다.
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", file_content),  # 시스템 메시지에 로드한 프롬프트를 넣습니다.
    ("user", "Context: {context}\\n\\nQuestion: {question}")  # 사용자 메시지에는 질문과 문맥을 넣습니다.
])

# 중간 결과를 출력하는 디버그 클래스입니다.
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)  # 부모 클래스의 invoke 메서드를 호출합니다.
        print("Debug Output:", output)  # 출력된 결과를 디버그용으로 출력합니다.
        return output  # 결과를 반환합니다.

# 문서 리스트를 텍스트로 변환하는 클래스입니다.
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):
        # 입력받은 문서들을 텍스트로 결합합니다.
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,  # 검색된 문서들을 가져오는 retriever
    "question": DebugPassThrough()  # 사용자 질문을 디버그 패스스로 출력하는 클래스
}  | DebugPassThrough() | ContextToText() | contextual_prompt | model  # 각 단계를 차례대로 연결합니다.

# 결과를 파일에 저장하기 위해 파일을 엽니다.
with open("C:/Users/USER/Desktop/RAG과제/AI챗봇/Results/prompt_result.txt", 'a', encoding='utf-8') as f:
    while True:  # 무한 루프를 시작합니다.
        print("========================")
        query = input("질문을 입력하세요: ")  # 사용자로부터 질문을 입력받습니다.

        if query == "quit":  # 'quit'를 입력하면 종료합니다.
            break

        # 입력된 질문을 기반으로 RAG 체인에서 결과를 가져옵니다.
        response = rag_chain_debug.invoke(query)

        # 최종 응답을 출력합니다.
        print("Final Response:")
        print(response.content)

        # 파일에 질문과 응답을 저장합니다.
        f.write("===============\n")
        f.write(f"질문: {query}\n")
        f.write(f"응답: {response.content}\n\n")

C:\Users\USER\AppData\Local\Temp\ipykernel_7056\1757506352.py:41: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


system
You are an expert Prompt Writer for Large Language Models.

human
Your goal is to improve the prompt given below for {question} :

--------------------

Prompt: {context}

--------------------

Here are several tips on writing great prompts:

-------

Start the prompt by stating that it is an expert in the subject.
Put instructions at the beginning of the prompt and use ### or to separate the instruction and context 
Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc 

---------

Here's an example of a great prompt:
As a master YouTube content creator, develop an engaging script that revolves around the theme of "Exploring Ancient Ruins."
Your script should encompass exciting discoveries, historical insights, and a sense of adventure.
Include a mix of on-screen narration, engaging visuals, and possibly interactions with co-hosts or experts.
The script should ideally result in a video of around 10-15 minutes, provid